In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import scipy.optimize as sco
import json

In [2]:
tickers = ['AMZN', 'BA', 'CAT', 'GOOGL', 'GS', 'NKE', 'NVDA', 'SOFI', 'TSLA', 'UNH']

In [3]:
# Parameters
train_start = "2024-01-17"
test_start = "2024-03-01"
test_end = "2025-01-17"
initial_capital = 50000
lookback_days = 21

# Download both Close and Open prices
data = yf.download(tickers, start=train_start, end=test_end, progress=False)
close = data['Close'].dropna()
openp = data['Open'].dropna()

# Log returns (from close prices)
log_returns = np.log(close / close.shift(1)).dropna()

# Generate monthly rebalance dates starting from test_start
rebalance_dates = pd.date_range(start=test_start, end=test_end, freq='MS')  # Month Start

# Map to actual trading days (forward-fill if it's not a trading day)
rebalance_dates = [close.index[close.index.get_indexer([d], method='bfill')[0]] for d in rebalance_dates]

# Init
weights_per_month = {}
shares_held = pd.Series(0, index=tickers)

for date in rebalance_dates:
    # Ensure date exists in index
    if date not in close.index:
        date = close.index[close.index.get_indexer([date], method='bfill')[0]]
    
    end_idx = close.index.get_loc(date)
    start_idx = end_idx - lookback_days
    if start_idx < 0:
        continue

    # Get past data window (only up to yesterday)
    window_returns = log_returns.iloc[start_idx:end_idx]
    mean_returns = window_returns.mean() * 252
    cov_matrix = window_returns.cov() * 252

    # Define Sharpe Ratio optimizer
    def neg_sharpe(weights):
        port_return = np.dot(weights, mean_returns.values)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix.values, weights)))
        return -port_return / port_vol

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = [(0.01, 0.3)] * len(tickers)
    init_guess = np.array([1 / len(tickers)] * len(tickers))

    result = sco.minimize(neg_sharpe, init_guess, method='SLSQP',
                          bounds=bounds, constraints=constraints)
    if not result.success:
        continue

    weights = pd.Series(result.x, index=tickers)

    # Log weights
    weights_per_month[pd.to_datetime(date).strftime("%Y-%m-%d")] = weights.round(4).to_dict()
    
# Weights per Month
print(" Diversified Weights per Month:")
print(json.dumps(weights_per_month, indent=2))


YF.download() has changed argument auto_adjust default to True
 Diversified Weights per Month:
{
  "2024-03-01": {
    "AMZN": 0.1364,
    "BA": 0.01,
    "CAT": 0.3,
    "GOOGL": 0.01,
    "GS": 0.01,
    "NKE": 0.0286,
    "NVDA": 0.2156,
    "SOFI": 0.2012,
    "TSLA": 0.0783,
    "UNH": 0.01
  },
  "2024-04-01": {
    "AMZN": 0.01,
    "BA": 0.01,
    "CAT": 0.3,
    "GOOGL": 0.3,
    "GS": 0.2114,
    "NKE": 0.01,
    "NVDA": 0.1286,
    "SOFI": 0.01,
    "TSLA": 0.01,
    "UNH": 0.01
  },
  "2024-05-01": {
    "AMZN": 0.01,
    "BA": 0.01,
    "CAT": 0.01,
    "GOOGL": 0.2118,
    "GS": 0.3,
    "NKE": 0.01,
    "NVDA": 0.01,
    "SOFI": 0.01,
    "TSLA": 0.1282,
    "UNH": 0.3
  },
  "2024-06-03": {
    "AMZN": 0.01,
    "BA": 0.1288,
    "CAT": 0.0353,
    "GOOGL": 0.01,
    "GS": 0.2566,
    "NKE": 0.247,
    "NVDA": 0.2823,
    "SOFI": 0.01,
    "TSLA": 0.01,
    "UNH": 0.01
  },
  "2024-07-01": {
    "AMZN": 0.1916,
    "BA": 0.01,
    "CAT": 0.01,
    "GOOGL": 0.1965,
    "

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  wa

In [4]:
predictions = {}
for stock in tickers:
    predictions[stock] = pd.read_csv('Predictions of ' + stock + '.csv', parse_dates=True, index_col=0)
    
def calculate_portfolio_daily_report():
    """
    Computes a daily portfolio value report for the test period (from 2024-03-01 to 2025-01-16)
    using the following methodology:
    
    1. On the first test day (or the next available trading day if 2024-03-01 isn't in the data),
       allocate the current portfolio value according to the weights provided for that period.
    2. For each day in the period until the next rebalancing date, update each stock's 
       allocated capital using the ratio of its predicted cumulative return on that day to the
       cumulative return at the period's start.
    3. On each day, the total portfolio value is computed by summing all per-stock values plus any
       uninvested capital.
    4. At each rebalancing day (adjusted to an actual trading day), the portfolio value is updated,
       and new allocations are made for the next period.
    
    This function returns a DataFrame indexed by day with detailed portfolio values.
    """
    portfolio_value = initial_capital
    rebalance_dates_sorted = list(weights_per_month.keys())
    daily_records = []
    
    # Assume all predictions share the same index of trading dates:
    pred_index = pd.to_datetime(predictions[tickers[0]].index)
    
    for i in range(len(rebalance_dates_sorted)):
        # Convert the desired rebalance date to datetime and adjust to the next trading day if needed.
        desired_start = pd.to_datetime(rebalance_dates_sorted[i])
        period_start = pred_index[pred_index.get_indexer([desired_start], method='bfill')[0]]
        
        # Determine the desired period end: if it's the last period, set to test_end; otherwise, next rebalance date.
        if i == len(rebalance_dates_sorted) - 1:
            desired_end = pd.to_datetime('2025-01-16')
        else:
            desired_end = pd.to_datetime(rebalance_dates_sorted[i+1])
        period_end = pred_index[pred_index.get_indexer([desired_end], method='bfill')[0]]
        
        # Create a daily date range for the current rebalancing period (using calendar days)
        # We later assume predictions exist only on trading days.
        period_days = pd.date_range(start=period_start, end=period_end, freq='D')
        
        # Get the weights for this period and compute initial allocation for each stock.
        weights = weights_per_month[rebalance_dates_sorted[i]]
        allocated_capital = {stock: weights[stock] * portfolio_value for stock in tickers}
        
        for d in period_days:
            day_stock_values = {}
            # For each stock, update the value using its predicted cumulative return
            for stock in tickers:
                # We adjust 'd' to the next available trading day if not present in predictions.
                try:
                    cum_return_day = predictions[stock]['Cumulative_Return_Strategy'].loc[d]
                except KeyError:
                    available_dates = pd.to_datetime(predictions[stock].index)
                    d_adjusted = available_dates[available_dates.get_indexer([d], method='bfill')[0]]
                    cum_return_day = predictions[stock]['Cumulative_Return_Strategy'].loc[d_adjusted]
                    
                # Get the cumulative return at the period start (which, by construction, exists)
                cum_return_start = predictions[stock]['Cumulative_Return_Strategy'].loc[period_start]
                # Compute the multiplicative growth factor for the day.
                factor = cum_return_day / cum_return_start
                
                day_stock_values[stock] = allocated_capital[stock] * factor
            
            # Calculate total invested from allocations (should roughly equal portfolio_value)
            invested = sum(allocated_capital.values())
            # Uninvested capital (if any, typically zero if weights sum exactly to 1)
            uninvested = portfolio_value - invested
            daily_total = sum(day_stock_values.values()) + uninvested
            
            record = {'Date': d,
                      'Total_Portfolio_Value': daily_total,
                      'Uninvested_Capital': uninvested}
            for stock in tickers:
                record[f'{stock}_Value'] = round(day_stock_values.get(stock, 0), 2)
            daily_records.append(record)
        
        # At the end of the period, update the portfolio value for the next period.
        portfolio_value = daily_records[-1]['Total_Portfolio_Value']
    
    df_daily_report = pd.DataFrame(daily_records).set_index('Date')
    return df_daily_report

# Generate and display the daily portfolio report.
portfolio_daily_report = calculate_portfolio_daily_report()
display("Portfolio Daily Report", portfolio_daily_report)
portfolio_daily_report.to_csv("portfolio_daily_report.csv")

'Portfolio Daily Report'

,Total_Portfolio_Value,Uninvested_Capital,AMZN_Value,BA_Value,CAT_Value,GOOGL_Value,GS_Value,NKE_Value,NVDA_Value,SOFI_Value,TSLA_Value,UNH_Value
Date,,,,,,,,,,,,
2024-03-01,50000.000000,-5.000000e+00,6820.00,500.00,15000.00,500.00,500.00,1430.00,10780.00,10060.00,3915.00,500.00
2024-03-02,50142.639185,-5.000000e+00,6832.25,499.33,15086.87,486.18,505.35,1446.71,10586.22,10297.56,3915.00,492.18
2024-03-03,50142.639185,-5.000000e+00,6832.25,499.33,15086.87,486.18,505.35,1446.71,10586.22,10297.56,3915.00,492.18
2024-03-04,50142.639185,-5.000000e+00,6832.25,499.33,15086.87,486.18,505.35,1446.71,10586.22,10297.56,3915.00,492.18
2024-03-05,50820.238655,-5.000000e+00,6898.81,498.58,14858.78,483.70,503.62,1455.73,10540.83,11186.92,3915.00,483.27
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-12,86242.357576,1.455192e-11,26006.71,25455.03,870.41,26131.90,834.54,873.33,3509.66,799.20,834.99,926.58
2025-01-13,86242.357576,1.455192e-11,26006.71,25455.03,870.41,26131.90,834.54,873.33,3509.66,799.20,834.99,926.58
2025-01-14,86457.252325,1.455192e-11,26048.37,25724.13,892.19,25947.21,847.26,878.90,3529.02,823.63,842.19,924.35


In [ ]:
# Final Portfolio Value: Rebalanced Strategy
final_rebalanced_value = portfolio_daily_report['Total_Portfolio_Value'].iloc[-1]
print(f"Final Portfolio Value (Rebalanced Strategy): ${final_rebalanced_value:,.2f}")


# Final Portfolio Value: Equal Allocation using Respective Strategy for Each Stock
start_date = '2024-03-01'
end_date = '2025-01-16'
equal_alloc = initial_capital / len(tickers) 

total_value_strategy_hold = 0
for stock in tickers:
    cum_return = predictions[stock]['Cumulative_Return_Strategy']
    try:
        # Growth factor: how much the strategy's cumulative return has multiplied from start to end.
        growth = cum_return.loc[end_date] / cum_return.loc[start_date]
    except KeyError:
        print(f"Missing cumulative return data for {stock} on {start_date} or {end_date}.")
        continue
    final_value = equal_alloc * growth
    total_value_strategy_hold += final_value

print(f"Final Portfolio Value (Equal Allocation, Strategy Cum Returns): ${total_value_strategy_hold:,.2f}")

# Final Portfolio Value: Equal-Weighted Buy-and-Hold using Actual Close Prices
total_value_buy_hold = 0
for stock in tickers:
    try:
        price_start = close[stock].loc[start_date]
        price_end = close[stock].loc[end_date]
    except KeyError:
        print(f"Missing trading day data for {stock} on {start_date} or {end_date}.")
        continue
    growth = price_end / price_start
    final_value = equal_alloc * growth
    total_value_buy_hold += final_value

print(f"Final Portfolio Value (Equal Allocation, Buy-and-Hold): ${total_value_buy_hold:,.2f}")

Final Portfolio Value (Rebalanced Strategy): $86,465.38
Final Portfolio Value (Equal Allocation, Strategy Cum Returns): $86,502.59
Final Portfolio Value (Equal Allocation, Buy-and-Hold): $67,449.19
